In [1]:
include("standard_hopfield.jl")
using .SH, Statistics, LinearAlgebra, Plots
using DelimitedFiles, Random

### Without simulated annealing

Now we start from a random pattern and we try to minimize the energy: we would like to end up in a stored pattern.

In [76]:
M = 10
N = 100
ξ = generate_patterns(M, N)
J = store(ξ)
nsweeps = 100

success = 0
ntrials = 100

#σ = init_pattern(N)
for _ in 1:ntrials
    σ = init_pattern(N)
    σ_rec = monte_carlo(J, σ; nsweeps = nsweeps, earlystop = 0, β = 10^6)
    overlaps = (σ_rec' * ξ)./N
    if findmax(overlaps)[1] >= 0.95
        success += 1
    end
end

success


16

### With simulated annealing

In [61]:
M = 10
N = 100
ξ = generate_patterns(M, N)
J = store(ξ)
nsweeps = 100


100